In [204]:

import os
import sys

from neural_classificator.classificator_validation import create_and_train_classifier

sys.path.append("../")
import cv2  # OpenCV
import matplotlib.pyplot as plt
import numpy as np
from config import DATA_PATH, RAW_DATA_PATH
%matplotlib inline

In [205]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)


def display_image(image):
    plt.figure()
    plt.imshow(image)

### HOG

In [206]:
# Create HOG classifier

classifier = create_and_train_classifier("HOG")

### HOUGH

In [207]:
def detect_lines(gray_img):
    edges_img = cv2.Canny(gray_img, 50, 150, apertureSize=3)
    # plt.imshow(edges_img, "gray")

    min_line_length = 200
    lines = cv2.HoughLinesP(image=edges_img, rho=1, theta=np.pi / 180, threshold=10, lines=np.array([]),
                            minLineLength=min_line_length, maxLineGap=20)

    lines[:, :, 1] = gray_img.shape[0] - lines[:, :, 1]
    lines[:, :, 3] = gray_img.shape[0] - lines[:, :, 3]

    img_with_lines = gray_img.copy()

    # Draw lines on the image
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)  # (0, 255, 0) is the color (green), 2 is the thickness

    # Display the image with lines using matplotlib (optional)
    plt.imshow(img_with_lines, cmap='gray')
    plt.show()

    lines = np.vstack([lines, [
        [[0, 1, 719, 1]],
        [[0, 721, 719, 721]],
        [[-1, 1, -1, 720]],
        [[719, 1, 719, 720]]
    ]])
    return lines


In [208]:
def check_fields_similarity(frame_field, next_frame_field, classifier, p=False):
    frame_field = cv2.resize(frame_field, (90, 90), interpolation=cv2.INTER_NEAREST)
    next_frame_field = cv2.resize(next_frame_field, (90, 90), interpolation=cv2.INTER_NEAREST)
    f1 = classifier.predict([frame_field])
    f2 = classifier.predict([next_frame_field])
    return f1[0], f2[0]


### operacije

In [209]:
def invert(similarities):
    inverse = []
    for i in range(8):
        for j in range(8):
            inverse.append(similarities[j * 8 + i])
    return inverse

In [210]:
def change_fen_row(row, idx, char):
    full_row = ''
    for c in row:
        if '9' > c > '0':
            full_row += 'e' * int(c)
        else:
            full_row += c
    full_row = full_row[:idx] + char + full_row[idx + 1:]
    row = ''
    cons_e = 0
    for c in full_row:
        if c != 'e':
            if cons_e != 0:
                row += str(cons_e)
                cons_e = 0
            row += c
        else:
            cons_e += 1
    if cons_e != 0:
        row += str(cons_e)
    return row

In [211]:
def get_fen_format(fem, figure, start_iter, end_iter, castling, en_passant=None):
    # rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w
    fem_table = fem[:-2]
    next_move = chr(217 - ord(fem[-1]))
    rows = fem_table.split('/')
    if figure in ['k', 'K'] and abs(start_iter // 8 - end_iter // 8) > 1:
        #king
        rows[start_iter % 8] = change_fen_row(rows[start_iter % 8], start_iter // 8, 'e')
        rows[end_iter % 8] = change_fen_row(rows[end_iter % 8], end_iter // 8, figure)
        #rook
        rows[start_iter % 8] = change_fen_row(rows[start_iter % 8], 0 if end_iter < start_iter else 7, 'e')
        rows[end_iter % 8] = change_fen_row(rows[end_iter % 8],
                                            end_iter // 8 + 1 if end_iter < start_iter else end_iter // 8 - 1,
                                            'r' if figure == 'k' else 'R')
        if start_iter%8==0:
            castling=castling[0:2]+'--'
        elif start_iter%8==7:
            castling='--'+castling[2:]
    else:
        if en_passant:
            rows[en_passant % 8] = change_fen_row(rows[en_passant %8], en_passant//8, 'e')
        rows[start_iter % 8] = change_fen_row(rows[start_iter % 8], start_iter // 8, 'e')
        rows[end_iter % 8] = change_fen_row(rows[end_iter % 8], end_iter // 8, figure)
    new_fem = ''
    for i in range(8):
        new_fem += rows[i]
        if i != 7:
            new_fem += '/'
    new_fem += ' ' + next_move
    return new_fem, castling

### main

In [212]:
def detect_moves(video_path, start_pos, lines, classifier):
    cap = cv2.VideoCapture(video_path)
    cap.set(1, 0)
    next_frame = start_pos
    frame=next_frame
    grabbed, next_next_frame = cap.read()
    # lines.append()
    moves = []
    frames = 0
    moves_count = 2
    fem = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w"
    castling = 'KQkq'
    while True:

        frame = next_frame
        next_frame = next_next_frame

        grabbed, next_next_frame= cap.read()
        if not grabbed:
            break
        
        frames += 1

        half_size = int(len(lines) / 2)
        iter = 0

        similarities = []
        differences = []
        for vert_line_ind in range(half_size - 1):
            for hor_line_ind in range(half_size - 1):
                x1 = lines[half_size + hor_line_ind][0][0] + 1
                x2 = lines[half_size + hor_line_ind + 1][0][0] + 1
                y1 = lines[vert_line_ind][0][1] - 1
                y2 = lines[vert_line_ind + 1][0][1] - 1
                f1, f2 = check_fields_similarity(frame[x1:x2, y1:y2], next_frame[x1:x2, y1:y2], classifier)
                similarities.append(f1 == f2)
                if f1 != f2:
                    differences.append((f1, f2, iter))

                iter += 1
        promotion=''
        en_passant=None
        if len(differences) == 2:
            if differences[0][1] == 'e':
                figure = differences[1][1]
                if differences[1][1]!=differences[0][0]:
                    promotion=figure.lower()
                start_iter = differences[0][2]
                start_field = chr(ord('a') + differences[0][2] // 8) + '' + str(8 - differences[0][2] % 8)
                end_iter = differences[1][2]
                end_field = chr(ord('a') + differences[1][2] // 8) + '' + str(8 - differences[1][2] % 8)
            elif differences[1][1] == 'e':
                figure = differences[0][1]
                if differences[0][1]!=differences[1][0]:
                    promotion=figure.lower()
                start_iter = differences[1][2]
                start_field = chr(ord('a') + differences[1][2] // 8) + '' + str(8 - differences[1][2] % 8)
                end_iter = differences[0][2]
                end_field = chr(ord('a') + differences[0][2] // 8) + '' + str(8 - differences[0][2] % 8)

        if len(differences) == 3:
            index_of_end_field = next((index for index, tup in enumerate(differences) if tup[0] == 'e'), None)
            figure=differences[index_of_end_field][1]
            index_of_start_field=next((index for index, tup in enumerate(differences) if tup[0] == figure), None)
            
            start_iter=differences[index_of_start_field][2]
            end_iter=differences[index_of_end_field][2]
            en_passant=differences[3-index_of_start_field-index_of_end_field][2]

            start_field = chr(ord('a') + differences[index_of_start_field][2] // 8) + '' + str(8 - differences[index_of_start_field][2] % 8)
            end_field = chr(ord('a') + differences[index_of_end_field][2] // 8) + '' + str(8 - differences[index_of_end_field][2] % 8)

            # [('P', 'e', 35), ('e', 'P', 42), ('p', 'e', 43)]
        if len(differences) == 4:
            for i in range(4):
                if differences[i][0] in ['k', 'K'] and differences[i][1] == 'e':
                    figure = differences[i][0]
                    start_iter = differences[i][2]
                    start_field = chr(ord('a') + differences[i][2] // 8) + '' + str(8 - differences[i][2] % 8)
                elif differences[i][0] == 'e' and differences[i][1] in ['k', 'K']:
                    end_iter = differences[i][2]
                    end_field = chr(ord('a') + differences[i][2] // 8) + '' + str(8 - differences[i][2] % 8)

        if 0 < len(differences) < 5:
            similarities = invert(similarities)
            fem, castling = get_fen_format(fem, figure, start_iter, end_iter, castling, en_passant)
            moves.append((start_field + end_field+promotion, fem + ' ' + castling + ' - 0 ' + str(moves_count // 2)))
            moves_count += 1

    return moves



In [213]:
def save_data(positions, moves, video):
    """
    Save moves and positions to a .npy file.
    """
    # Reshape moves and positions into 2D arrays (columns)
    positions_array = np.array(positions).reshape(-1, 1)
    moves_array = np.array(moves).reshape(-1, 1)
    positions_and_moves = np.concatenate((positions_array, moves_array), axis=1)

    np.save(f"{RAW_DATA_PATH}/{video}.npy", positions_and_moves)
    print("Saved successfully")

In [214]:
start_pos_img = load_image(f'{DATA_PATH}/start1.png')
output_file_path = f'{DATA_PATH}/moves.txt'
lines = detect_lines(start_pos_img)
lines = sorted(lines, key=lambda line: (line[0][0], line[0][1]))
first_line = lines.pop(0)

lines.insert(9, first_line)

# video_dir = f'{DATA_PATH}/videos100/'
# with open(output_file_path, 'w'):
#     pass

# with open(output_file_path, 'a') as file:
#     for video in sorted(os.listdir(video_dir)):
#         # video="board (10).gif"
#         print(video)
#         video_path = os.path.join(video_dir, video)
#         positions_and_moves = detect_moves(video_path, start_pos_img, lines, classifier)

#         positions = []
#         moves = []

#         for (move, position) in positions_and_moves:
#             positions.append(position)
#             moves.append(move)

#         save_data(positions, moves, video.split('.')[0])


board (1).gif
Saved successfully
board (10).gif
Saved successfully
board (100).gif
Saved successfully
board (101).gif
Saved successfully
board (102).gif
Saved successfully
board (103).gif
Saved successfully
board (11).gif
Saved successfully
board (12).gif
Saved successfully
board (13).gif
Saved successfully
board (14).gif
Saved successfully
board (15).gif
Saved successfully
board (16).gif
Saved successfully
board (17).gif
Saved successfully
board (18).gif
Saved successfully
board (19).gif
Saved successfully
board (2).gif
Saved successfully
board (20).gif
Saved successfully
board (21).gif
Saved successfully
board (22).gif
Saved successfully
board (23).gif
Saved successfully
board (24).gif
Saved successfully
board (25).gif
Saved successfully
board (26).gif
Saved successfully
board (27).gif
Saved successfully
board (28).gif
Saved successfully
board (29).gif
Saved successfully
board (3).gif
Saved successfully
board (30).gif
Saved successfully
board (31).gif
Saved successfully
board (32).gi